In [23]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("source/memory.pdf")
doc = loader.load()

In [47]:
print(len(doc))
print(len(doc[0].page_content))
print(len(doc[17].page_content))


18
2012
1804


In [98]:
# 페이지 단위 분할 대신, 500토큰 단위로 더 잘게 자른 실
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# texts = []
#
# for i in range(len(doc)):
#     texts += text_splitter.create_documents([doc[0].page_content])
#
# len(texts)
# len(texts[2].page_content)
texts = doc

In [52]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
import pinecone #쓰긴 편하지만, 로컬 벡터 베이터베이스를 사용하는게 여러모로 더 나아보인다.

/Users/lee/db-connection-test/venv/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [56]:
pinecone.init( #pinecone.io에 들어가서, 회원가입 하면 API Keys 확인할 수 있다.
    api_key="api 키 넣는 곳",
    environment="us-west4-gcp"
)
index_name = "osmemory"

In [54]:
embeddings = OpenAIEmbeddings(openai_api_key="api 키 넣는 곳")

In [99]:
if index_name not in pinecone.list_indexes(): #db만들기. 웹페이지에서 create index할 필요 없다.
    pinecone.create_index(index_name, dimension=1536)

index = pinecone.Index(index_name)

In [100]:
docsearch = Pinecone.from_texts( #openai ada2로 임베딩
    [t.page_content for t in texts], embeddings, index_name=index_name
)

In [60]:
from langchain.llms import OpenAI #이제, 뽑아온 db정보를 바탕으로 검색해보자
from langchain.chains.question_answering import load_qa_chain

In [68]:
llm = OpenAI(temperature=0, openai_api_key="api 키 넣는 곳")
chain = load_qa_chain(llm, chain_type="stuff")

In [114]:
query = "Why do we have to use swap policy?"
docs = docsearch.similarity_search(query, include_metadata=True, k=2) #가장 유사도 높은 문서 2개, 약 1300토큰

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: The server is currently overloaded with other requests. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists..


In [115]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    answer = chain.run(input_documents=docs, question=query)
    print(f"Total tokens : {cb.total_tokens}")

Total tokens : 1740


In [116]:
answer

' Swap policy is used to minimize the number of cache misses, i.e., to minimize the number of times that we have to fetch a page from disk. It is also used to maximize the number of cache hits, i.e., the number of times a page that is accessed is found in memory. This helps to calculate the average memory access time (AMAT) for a program.'